<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/develop/dimathina_retriever_train_dense_vdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install pinecone-client
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

wiki_validation_dataset = load_dataset('mandarjoshi/trivia_qa', 'rc.wikipedia', split='validation')

README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

train-00000-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

train-00001-of-00007.parquet:   0%|          | 0.00/261M [00:00<?, ?B/s]

train-00002-of-00007.parquet:   0%|          | 0.00/319M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/235M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/221M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61888 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7993 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7701 [00:00<?, ? examples/s]

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
unique_contexts = []
unique_ids = []

print(wiki_validation_dataset)
# make list of IDs that represent only first instance of
# each context
for row in wiki_validation_dataset:
    if row['entity_pages']['wiki_context'] not in unique_contexts:
        unique_contexts.append(row['entity_pages']['wiki_context'])
        unique_ids.append(row['question_id'])

# now filter out any samples that aren't included in unique IDs
wiki_validation_dataset = wiki_validation_dataset.filter(lambda x: True if x['question_id'] in unique_ids else False)
wiki_validation_dataset

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
    num_rows: 7993
})


Filter:   0%|          | 0/7993 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
    num_rows: 7399
})

In [15]:
# and now encode the unique contexts

def encode_function(examples):
    encodings = []
    for example in examples['entity_pages']:  # Iterate through individual examples
        wiki_contexts = example['wiki_context']
        # Ensure 'wiki_context' is a list, even if it's a single element
        if not isinstance(wiki_contexts, list):
            wiki_contexts = [wiki_contexts]

        # Join the list of contexts into a single string
        context = " ".join(wiki_contexts)
        encodings.append(model.encode(context))

    return {'encoding': encodings}

wiki_validation_dataset = wiki_validation_dataset.map(encode_function, batched=True, batch_size=4)

wiki_validation_dataset

Map:   0%|          | 0/7399 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer', 'encoding'],
    num_rows: 7399
})

In [16]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata


API_KEY = userdata.get('PINECONE_TOKEN')

pc = Pinecone(
    api_key=API_KEY
)

In [17]:
spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1",
  )

# check if index already exists, if not we create it
if 'wiki-index' not in pc.list_indexes():
    pc.create_index(
        name='wiki-index', dimension=model.get_sentence_embedding_dimension(), metric='cosine',spec=spec
    )

# we use this to get required index dims
model.get_sentence_embedding_dimension()

384

In [18]:
# initialize connection to the new index
index = pc.Index('wiki-index')

from tqdm.auto import tqdm  # progress bar

upserts = []
for v in wiki_validation_dataset:
    # Truncate the wiki_context to a maximum of 2000 characters
    truncated_context = str(v['entity_pages']['wiki_context'])[:3000]
    upserts.append((v['question_id'], [float(x) for x in v['encoding']], {'bytes': truncated_context}))

# now upsert in chunks
for i in tqdm(range(0, len(upserts), 5)):
    i_end = i + 5
    if i_end > len(upserts): i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])

  0%|          | 0/1480 [00:00<?, ?it/s]

In [19]:
query = "When was Baron Andrew Lloyd Webber born?"
xq = model.encode([query]).tolist()
xc = index.query(vector=xq, top_k=2, include_metadata=True)
xc

{'matches': [{'id': 'tc_33',
              'metadata': {'bytes': "['Andrew Lloyd Webber, Baron "
                                    'Lloyd-Webber   (born 22 March 1948) is an '
                                    'English composer and impresario of '
                                    'musical theatre. \\n\\nSeveral of his '
                                    'musicals have run for more than a decade '
                                    'both in the West End and on Broadway. He '
                                    'has composed 13 musicals, a song cycle, a '
                                    'set of variations, two film scores, and a '
                                    'Latin Requiem Mass. Several of his songs '
                                    'have been widely recorded and were hits '
                                    'outside of their parent musicals, notably '
                                    '"The Music of the Night" from The Phantom '
                            